In [1]:
import os
import sys

import numpy as np
import pandas as pd
from sqlalchemy import create_engine


sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('../..'))))
from strategy import EquityStrategy
from tester import Tester


base_path = os.path.abspath('../../..')
data_path = os.path.join(base_path, 'data')
database_path = os.path.join(data_path, 'database')
strategy_path = os.path.join(base_path, 'strategy')
check_path = os.path.join(strategy_path, 'check')
sys.path.append(strategy_path)

In [2]:
from strategy import CommodityStrategy, EquityStrategy, IRStrategy, EmergingStrategy
from tester import Tester

In [3]:
eindex_path = os.path.join(data_path, 'totindex.csv')
eindex1_path = os.path.join(data_path, 'priceindex.csv')
efuture_path = os.path.join(data_path, 'FutGenratio1.csv')

In [4]:
eindex_path = os.path.join(data_path, 'totindex.csv')
eindex1_path = os.path.join(data_path, 'priceindex.csv')
efuture_path = os.path.join(data_path, 'FutGenratio1.csv')

eindex = pd.read_csv(eindex_path, header=0, index_col=0, parse_dates=True)
eindex1 = pd.read_csv(eindex1_path, header=0, index_col=0, parse_dates=True)
efut = pd.read_csv(efuture_path, header=0, index_col=0, parse_dates=True)

eindex1 = eindex1.loc[:'2008']
ERetp = eindex1.pct_change(1).iloc[1:]
ERett = eindex.pct_change(1).iloc[1:]
fRet = efut.pct_change(1).iloc[1:]

compRet = ERett
compRet[compRet.isna()] = ERetp[compRet.isna()]
ERet1 = compRet
ERet1.drop_duplicates(inplace=True)
fRet.drop_duplicates(inplace=True)
ERet = pd.concat([ERet1.loc[:'2007-12-31'], fRet.loc['2008-01-01':]], axis=0)
ERet = ERet.fillna(0)
Eindex = (1 + ERet).cumprod()
Eindex = Eindex / Eindex.iloc[0]

In [5]:
minobs1=12
minobs=60
nopos=0.4
cs=0.35

In [104]:
i = 0

In [105]:
ivol = pd.read_csv(os.path.join(data_path, "ivol.csv"), index_col=0, parse_dates=True)
ivol.index.name = 'tdate'
ivol.columns.name = 'ticker'

In [106]:
if i == 0:
    RET = ERet.loc[:, ['SPX', 'TSX', 'FTSE', 'DAX', 'CAC', 'SMI', 'MIB', 'IBEX', 'OMX', 'AEX']]
else:
    RET = ERet.loc[:, ['NKY', 'AS51', 'HSI', "SG"]]

index = Eindex.loc[:, RET.columns]

# 1. Load Data
ivol = ivol
ivol = ivol[[column for column in ivol.columns if column in index.columns]]
ivol = ivol[pd.to_datetime('2002-01-01'):]

In [107]:
ivol1 = (ivol - ivol.shift(1)).iloc[1:]
RV = (ivol1.T - ivol1.mean(axis=1).T).T
RVrank = RV.iloc[minobs1 - 1:] * 0

In [114]:
pctrank = lambda x: pd.Series(x).rank(pct=True).iloc[-1]
RVrank1 = RV.expanding().apply(pctrank, raw=True)  # it takes some time
RVrank1 = RVrank1.iloc[minobs1 - 1:minobs - 1]

In [116]:
pctrank = lambda x: pd.Series(x).rank(pct=True).iloc[-1]
RVrank2 = RV.rolling(minobs).apply(pctrank, raw=True)  # it takes some time
RVrank2 = RVrank2.iloc[minobs - 1:]
RVrank = pd.concat((RVrank1, RVrank2), axis=0)

In [15]:
truecount = np.round((RVrank.shape[-1] - RVrank.isna().sum(1)) * cs)
truecount = np.repeat(truecount.values, len(RVrank.columns)).reshape(RVrank.shape)

tiebreaker = RVrank.rolling(5).mean() * 0.0000001  # tie breaker what is this??
tiebreaker.iloc[:4] = 0
tied_RVrank = RVrank + tiebreaker

CSRV = tied_RVrank.rank(1, method='first')
CSRV1 = (CSRV.count(1).T + 1 - CSRV.T).T

CSRVpos = CSRV * 0

CSRVpos[CSRV <= truecount] = -1
CSRVpos[CSRV1 <= truecount] = 1
CSRV = CSRVpos.fillna(0)

TSRV = RVrank * 0

TSRV[RVrank > (nopos + (1 - nopos) / 2)] = 1
TSRV[RVrank < ((1 - nopos) / 2)] = -1
TSRV.fillna(0, inplace=True)

In [16]:
TSRV.head()

ticker,SPX,TSX,FTSE,DAX,CAC,SMI,MIB,IBEX,OMX,AEX
tdate,,,,,,,,,,
2003-01-31,1.0,1.0,1.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0
2003-02-28,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2003-03-31,-1.0,0.0,1.0,-1.0,1.0,0.0,0.0,0.0,-1.0,1.0
2003-04-30,1.0,1.0,-1.0,-1.0,-1.0,1.0,0.0,0.0,0.0,-1.0
2003-05-30,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.0


In [87]:
eql.TSRV.to_csv('../past/eql_tsrv_python.csv')

In [88]:
eql.CSRV.to_csv('../past/eql_csrv_python.csv')

In [98]:
eql_TSRV = pd.read_csv('../past/eql_tsrv.csv', index_col=[0])
eql_CSRV = pd.read_csv('../past/eql_csrv.csv', index_col=[0])

In [101]:
eql_TSRV.index = pd.to_datetime(eql_TSRV.index)
eql_CSRV.index = pd.to_datetime(eql_CSRV.index)

In [132]:
TSRV_DIFF = (eql_TSRV - eql.TSRV)

In [133]:
TSRV_DIFF[np.abs(TSRV_DIFF) > 0.01].count()

SPX     0
TSX     0
FTSE    0
DAX     0
CAC     0
SMI     0
MIB     0
IBEX    0
OMX     0
AEX     0
NKY     0
AS51    0
HSI     0
SG      0
dtype: int64

In [134]:
TSRV_DIFF.sum(axis=0)

SPX    -9.936496e-15
TSX     1.346701e-13
FTSE    1.331157e-13
DAX     1.542655e-13
CAC     8.587575e-14
SMI     1.713074e-13
MIB     7.044365e-14
IBEX    9.692247e-14
OMX     9.586776e-14
AEX     2.448042e-14
NKY     3.419487e-14
AS51    1.846856e-13
HSI     2.553513e-15
SG     -1.591505e-13
dtype: float64

In [135]:
CSRV_DIFF = (eql_CSRV - eql.CSRV)

In [145]:
CSRV_DIFF[np.abs(CSRV_DIFF) > 0.001].sum(axis=0)

SPX     -7.000000
TSX    -14.666667
FTSE     0.000000
DAX    -14.333333
CAC      0.000000
SMI      0.000000
MIB      7.000000
IBEX     0.000000
OMX     14.666667
AEX     14.333333
NKY      0.000000
AS51     0.000000
HSI      0.000000
SG       0.000000
dtype: float64